# Imports

In [2]:
import pandas as pd
import h5py
import numpy as np
import tensorflow as tf
import h5py
import os
from sklearn.metrics import accuracy_score
import ecg_plot
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,precision_recall_fscore_support

In [8]:
#Lanzar desde la carpeta LightX3ECGPrivate
import sys
sys.path.append('./automatic-ecg-diagnosis/')
# sys.path.append('./source/')
import predict

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

# Commands

In [2]:
!git clone https://github.com/antonior92/automatic-ecg-diagnosis.git

Cloning into 'automatic-ecg-diagnosis'...


In [7]:
#Comando windows
!py -m pip install tensorflow

^C


Defaulting to user installation because normal site-packages is not writeable


In [46]:
!cd

e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis


In [4]:
%cd automatic-ecg-diagnosis

e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis


In [2]:
!ls

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [11]:
!wget https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0 -O data.zip

--2023-05-02 18:21:58--  https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6024:18::a27d:4412
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: /s/raw/p3vd3plcbu9sf1o/data.zip [siguiendo]
--2023-05-02 18:21:58--  https://www.dropbox.com/s/raw/p3vd3plcbu9sf1o/data.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USPaOcJJ6qJM3mylVU6u86B3BR2k6jWUil7g29fotfKh0d6Kir86bKWE2Qd4ddmw2mvBrzpm5X-3y240SOomVSv8E0sCPCoBATC3cVbe2ZJF0MvPxg/file# [siguiendo]
--2023-05-02 18:21:59--  https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USP

In [12]:
!unzip data.zip

Archive:  data.zip
  inflating: data/ecg_tracings.hdf5  
replace data/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
!pip3 show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/tfg_luis_2223/.local/lib/python3.7/site-packages
Requires: google-pasta, astor, gast, keras-preprocessing, six, protobuf, wheel, keras-applications, absl-py, tensorboard, wrapt, grpcio, numpy, termcolor, tensorflow-estimator
Required-by: 


In [5]:
!python install -U pip

python: can't open file 'install': [Errno 2] No such file or directory


In [7]:
!python -v

# installing zipimport hook
import zipimport # builtin
# installed zipimport hook
# /usr/lib/python2.7/site.pyc matches /usr/lib/python2.7/site.py
import site # precompiled from /usr/lib/python2.7/site.pyc
# /usr/lib/python2.7/os.pyc matches /usr/lib/python2.7/os.py
import os # precompiled from /usr/lib/python2.7/os.pyc
import errno # builtin
import posix # builtin
# /usr/lib/python2.7/posixpath.pyc matches /usr/lib/python2.7/posixpath.py
import posixpath # precompiled from /usr/lib/python2.7/posixpath.pyc
# /usr/lib/python2.7/stat.pyc matches /usr/lib/python2.7/stat.py
import stat # precompiled from /usr/lib/python2.7/stat.pyc
# /usr/lib/python2.7/genericpath.pyc matches /usr/lib/python2.7/genericpath.py
import genericpath # precompiled from /usr/lib/python2.7/genericpath.pyc
# /usr/lib/python2.7/warnings.pyc matches /usr/lib/python2.7/warnings.py
import warnings # precompiled from /usr/lib/python2.7/warnings.pyc
# /usr/lib/python2.7/linecache.pyc matches /usr/lib/python2.7/linecache.

# Code

## crear hdf5

In [48]:
filename = 'data/data2.hdf5'
f = h5py.File(filename, 'w')

In [49]:
folder_path = 'CasosNumpy'
train = pd.read_csv("data/annotations/train.csv")
val = pd.read_csv("data/annotations/val.csv")
df = pd.concat([train,val],ignore_index=True,)
group_name = 'tracings'
shape = (len(df['id'].values),4096,12)
dset = f.create_dataset(group_name,shape=shape)

count = 0
for file_name in os.listdir(folder_path):
    if(file_name[:5] in df['id'].values):
        if file_name.endswith('.npy'):
            # Cargar el archivo numpy
            arr = np.load(os.path.join(folder_path, file_name))[:,:4096]
            if arr.shape != (12, 4096):
                new_arr = np.zeros((12, 4096))
                new_arr[:arr.shape[0], :arr.shape[1]] = arr
                arr = new_arr
            #arr = tf.convert_to_tensor(arr, dtype=tf.float32)
            dset[count, :, :] = arr.transpose()
            count+=1
            
# Guardar el arreglo numpy en el grupo

In [50]:
print(len(f[group_name]))
f.close()

6877


## Modificar val.csv a como lo tiene antonio

In [44]:
val = pd.read_csv("data/annotations/val.csv")
#val = val.drop(['id','age','sex','r_count','length'],axis=1)

train = pd.read_csv("data/annotations/train.csv")
#print(val)
#print(train)
final = pd.concat([train,val],ignore_index=True)
final = final.sort_values('id',)
final = final.reset_index()
final = final.drop(['index','id','age','sex','r_count','length'],axis=1)

print(final)

      label_0  label_1  label_2  label_3  label_4  label_5  label_6  label_7   
0           0        0        0        0        1        0        0        0  \
1           1        0        0        0        0        0        0        0   
2           0        1        0        0        0        0        0        0   
3           0        1        0        0        0        0        0        0   
4           0        0        0        0        0        0        1        0   
...       ...      ...      ...      ...      ...      ...      ...      ...   
6872        0        0        1        0        0        0        0        0   
6873        0        0        0        0        0        0        0        1   
6874        0        0        0        1        0        0        0        0   
6875        0        1        0        0        0        0        0        0   
6876        0        0        0        0        0        0        1        0   

      label_8  
0           0  
1      

In [45]:
final.to_csv("data/annotations/train_CPSC.csv",)


## Crear un npy que tenga todos los datos de entrenamiento y validación

In [42]:
folder_path = 'CasosNumpy'
train = pd.read_csv("data/annotations/train.csv")
val = pd.read_csv("data/annotations/val.csv")
df = pd.concat([train,val],ignore_index=True,)
finalArr = np.empty((0,5000,12))
for id in train['id'].values:
    arr = np.load(os.path.join(folder_path, f"{id}.npy"))[:,:5000]
    if arr.shape != (12, 5000):
        new_arr = np.zeros((12, 5000))
        new_arr[:arr.shape[0], :arr.shape[1]] = arr
        arr = new_arr
        print(id)
        print(arr.shape)
    finalArr = np.append(finalArr, [arr.transpose()],axis= 0)
finalArr.shape

KeyboardInterrupt: 

In [45]:
np.save("data_cpsc.npy",finalArr)

In [46]:
inp = np.load("data_cpsc.npy")
inp.shape

(6877, 4096, 12)

In [43]:
newdf = df.drop(['id','age','sex','r_count','length'],axis=1)
newdf

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8
0,0,1,0,0,1,0,0,0,0
1,0,1,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6872,0,0,0,0,1,0,0,0,0
6873,0,0,1,0,0,1,0,0,0
6874,0,0,1,0,0,0,0,0,0
6875,0,0,0,0,0,0,1,0,0


In [44]:
newdf.to_csv("train_cpsc.csv",index=False)

## Crear un npy para poder comprar los modelos

In [59]:
folder_path = 'CasosNumpy'
pred = pd.read_csv("data/annotations/train.csv")
finalArr = np.empty((len(pred),5000,12))
cont = 0
for id in pred['id'].values:
    arr = np.load(os.path.join(folder_path, f"{id}.npy"))[:,:5000]
    if arr.shape != (12, 5000):
        print(id)
        print(arr.shape)
        new_arr = np.zeros((12, 5000))
        new_arr[:arr.shape[0], :arr.shape[1]] = arr
        arr = new_arr
    finalArr[cont] = arr.transpose()
    cont += 1
finalArr.shape

A4229
(12, 4500)
A0240
(12, 4999)
A2663
(12, 4000)
A0798
(12, 4500)
A4085
(12, 4999)
A1872
(12, 4500)
A2695
(12, 4500)
A0626
(12, 4999)


(5501, 5000, 12)

In [60]:
np.save("data_train_cpsc.npy",finalArr)

## Modificar CSV de CPSC-2018 para las predicciones que hace antonio

In [8]:
## Ant: 1aAVb,RBBB,LBBB,SB,AF,ST
## CPSC: normal, AF,IAVB,LBBB,RBBB,PAC,PVC,STD,STE
train = pd.read_csv("data/annotations/train.csv")
val = pd.read_csv("data/annotations/val.csv")
pred = pd.read_csv("data/annotations/pred.csv")
df = pd.concat([train,val,pred],ignore_index=True,)

df = df.loc[
    (df['label_0'] == 1) |
    (df['label_1'] == 1) |
    (df['label_2'] == 1) |
    (df['label_3'] == 1) |
    (df['label_4'] == 1)]
df

,id,age,sex,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,r_count,length
0,A3483,75,0,0,1,0,0,1,0,0,0,0,19,5000
1,A1654,80,0,0,1,0,0,1,0,0,0,0,14,5000
2,A0345,77,1,0,0,0,0,1,0,0,0,0,13,5000
3,A3568,82,1,0,0,0,0,1,0,0,0,0,11,5000
4,A0370,74,0,0,1,0,0,0,0,0,0,0,22,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6887,A1213,65,0,0,0,0,1,0,0,0,0,0,12,5000
6888,A1754,75,1,0,0,0,1,0,0,0,0,0,13,5000
6889,A0345,77,1,0,0,0,0,1,0,0,0,0,13,5000
6890,A2701,35,1,0,0,0,0,1,0,0,0,0,13,5000


In [21]:
df_pred = df.drop(['id', 'age', 'sex', 'label_0', 'label_5', 'label_6', 'label_7', 'label_8', 'r_count','length'], axis=1)
df_pred.to_csv("pred_CPSC_antonizado.csv",index=False)

In [16]:
folder_path = 'Prueba/Prueba/'
finalArr = np.empty((len(df['id']),4096,12))
cont = 0
for id in df['id'].values:
    arr = np.load(os.path.join(folder_path, f"{id}.npy"))
    if arr.shape != (12, 4096):
        new_arr = np.zeros((12, 4096))
        new_arr[:arr.shape[0], :arr.shape[1]] = arr
        arr = new_arr
        print(id)
        print(arr.shape)
    finalArr[cont] = arr.transpose()
    cont += 1
finalArr.shape

(6877, 4096, 12)

In [18]:
np.save("data_cpsc_antonizado.npy",finalArr)

## Comprobar longitud de las muestras

In [14]:
folder_path = 'CasosNumpy'
train = pd.read_csv("data/annotations/train.csv")
val = pd.read_csv("data/annotations/val.csv")
df = pd.concat([train,val],ignore_index=True,)
count = 0
for id in train['id'].values:
    arr = np.load(os.path.join(folder_path, f"{id}.npy"))
    if arr.shape[1] >  8192:
        print(id)
        print(arr.shape)
        count += 1
print(train.shape)
print(count)

A1654
(12, 14000)
A0345
(12, 11000)
A2795
(12, 15500)
A1007
(12, 15000)
A5762
(12, 10000)
A2808
(12, 11177)
A4409
(12, 11500)
A2042
(12, 16818)
A5127
(12, 11500)
A6128
(12, 25927)
A4981
(12, 12378)
A5254
(12, 18000)
A6860
(12, 13000)
A4374
(12, 9644)
A4747
(12, 10461)
A4560
(12, 28161)
A0587
(12, 14500)
A5426
(12, 18500)
A0514
(12, 12000)
A0308
(12, 8316)
A2860
(12, 10000)
A5990
(12, 8258)
A2306
(12, 14949)
A3894
(12, 8500)
A1000
(12, 12000)
A3502
(12, 8424)
A3611
(12, 11500)
A4341
(12, 8777)
A5823
(12, 10000)
A6769
(12, 11000)
A6638
(12, 10000)
A2943
(12, 23305)
A3376
(12, 9937)
A5371
(12, 8695)
A2822
(12, 29500)
A4318
(12, 21718)
A0520
(12, 15000)
A4164
(12, 9500)
A4643
(12, 8500)
A3394
(12, 10500)
A6364
(12, 11000)
A1784
(12, 18316)
A1786
(12, 20145)
A0440
(12, 14000)
A3272
(12, 10000)
A1100
(12, 11500)
A3861
(12, 10500)
A4848
(12, 19000)
A5901
(12, 14000)
A1852
(12, 10000)
A6702
(12, 15924)
A2933
(12, 20000)
A0059
(12, 10000)
A0985
(12, 13161)
A1305
(12, 17505)
A0262
(12, 9000)
A49

## Otros

In [10]:
args= "data/data2.hdf5"
with h5py.File(args, "r") as f:
    x = np.array(f['tracings'])

np.size(x[1])
x.shape

(6877, 4096, 12)

In [11]:
np.save(file="data_numpy.npy",arr=x)

In [13]:
npy = np.load("./data_cpsc_antonizado.npy")[5501:]
npy.shape

(1376, 4096, 12)

In [14]:
np.save(file = 'data_val_cpsc_zado.npy',arr = npy)

In [39]:
def ecgPlot(data,sample, factor = 1):
    data = data* factor
    print(data.shape)
    #print(data)
    xml_leads = ['DI', 'DII', 'DIII', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    #ecg_plot.plot(data.T, sample_rate= sample,lead_index=xml_leads, title=f"{filename}",row_height=10)
    ecg_plot.plot_12(data.T, sample_rate= sample,lead_index=xml_leads, title=f"{filename}",columns=1)
    ecg_plot.save_as_png("ecg")

In [13]:
filename = './data/train.hdf5'
f = h5py.File(filename, 'r')

In [41]:
ecgPlot(f['tracings'][1],400)

(4096, 12)


In [84]:
path = "./data/annotations/pred.csv"
newpath = "./CompareFiles/pred.csv"
df = pd.read_csv(path)
df = df.drop(["id",'age','sex',"r_count","length"], axis = 1)
df.to_csv(newpath,index=False)

In [ ]:
df = pd.read_csv("./")

In [12]:
asd = np.load("./data_train_cpsc.npy")
asd.shape

(5501, 5000, 12)

# Train

In [21]:
!py train.py ./data/data.hdf5 ./data/annotations/train_CPSC.csv --val_split 0.2


54/54 [==============================] - 379s 7s/step - loss: -12790.8604 - val_loss: -69358.4297 - lr: 0.0010

86/86 [==============================] - 505s 6s/step - loss: -34734.5156 - val_loss: -184735.9688 - lr: 0.0010


In [31]:
!py train.py -h

usage: train.py [-h] [--val_split VAL_SPLIT] [--dataset_name DATASET_NAME]
                path_to_hdf5 path_to_csv

Train neural network.

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_csv           path to csv file containing annotations

optional arguments:
  -h, --help            show this help message and exit
  --val_split VAL_SPLIT
                        number between 0 and 1 determining how much of the
                        data is to be used for validation. The remaining is
                        used for validation. Default: 0.02
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings


In [9]:
!py train.py ./data_cpsc.npy ./data/annotations/train_CPSC.csv --val_split 0.0

^C


[1376 1377 1378 ... 6874 6875 6876]
[   0    1    2 ... 1373 1374 1375]
(6877, 4096, 12)

172/172 [==============================] - 596s 3s/step - loss: 0.2840 - val_loss: 0.4152 - lr: 0.0010


Traceback (most recent call last):
  File "e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis\train.py", line 81, in <module>
    
AttributeError: 'Functional' object has no attribute 'score'
Traceback (most recent call last):
  File "e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis\train.py", line 82, in <module>
    score = model.score(X_train,y_train)
            ^^^^^^^^^^^
AttributeError: 'Functional' object has no attribute 'score'


[1376 1377 1378 ... 6874 6875 6876]
[   0    1    2 ... 1373 1374 1375]
(6877, 4096, 12)

172/172 [==============================] - 700s 4s/step - loss: 0.2891 - val_loss: 0.2911 - lr: 0.0010


# Predict

In [19]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py data_cpsc_antonizado.npy model.hdf5 --output_file $PFOLDER/pred_cpsc_antonizado.npy


215/215 [==============================] - 83s 381ms/step
Output predictions saved


In [16]:
PFOLDER="./CompareFiles"

!python predict.py ./data_val_cpsc_zado.npy final_model_cpsc_zado.hdf5 --output_file $PFOLDER/pred_val_CPSC_zado.npy


43/43 [==============================] - 15s 346ms/step
Output predictions saved


In [30]:
!python predict.py -h

usage: predict.py [-h] [--dataset_name DATASET_NAME]
                  [--output_file OUTPUT_FILE] [-bs BS]
                  path_to_hdf5 path_to_model

Get performance on test set from hdf5

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_model         file containing training model.

optional arguments:
  -h, --help            show this help message and exit
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings
  --output_file OUTPUT_FILE
                        output csv file.
  -bs BS                Batch size.


# Comparar resultados de los modelos

In [19]:
predAnt = np.load("./CompareFiles/pred_val_CPSC_zado.npy")
print(predAnt.shape)
#predCPSC = pd.read_csv("train_cpsc.csv")
predCorrects = pd.read_csv("./CompareFiles/pred_val_CPSC.csv")
thresholdAnt = pd.read_csv('dnn_predicts/optimal_thresholds_best.csv')
#thresholdCPSC = pd.read_csv('dnn_predicts/optimal_thresholds_best.csv')

resultAnt = pd.DataFrame(data= [])
#resultCPSC = pd.DataFrame(data= predCPSC.values)

for pred in predAnt:
    data = pd.DataFrame([pred >= thresholdAnt['Threshold']])
    resultAnt = pd.concat([resultAnt,data])
resultAnt = resultAnt.replace({True:1,False:0})

accuracyAnt = accuracy_score(predCorrects.values,resultAnt.values)
print(accuracyAnt)

#accuracyCPSC = accuracy_score(predCorrects.values,resultCPSC.values)
#print(accuracyCPSC)

(1376, 9)
0.6984011627906976


In [18]:
val_prec, val_recall, val_f1, _ = precision_recall_fscore_support(
    predCorrects, resultAnt
    , average = None
)

df = pd.DataFrame(np.array([val_prec,val_recall,val_f1]).transpose(),columns=["Precision","Recall","F1"],index=["Normal","AF","I-AVB","LBBB","RBBB","PAC","PVC","STD","STE"])
df.loc['Average'] = df.mean()
df.index.names = ['Class']
print(df)

         Precision    Recall        F1
Class                                 
Normal    0.990489  0.997264  0.993865
AF        0.997953  0.994898  0.996423
I-AVB     0.998264  0.996534  0.997398
LBBB      1.000000  1.000000  1.000000
RBBB      0.997991  1.000000  0.998994
PAC       1.000000  0.979339  0.989562
PVC       0.996454  0.989437  0.992933
STD       0.995714  0.997139  0.996426
STE       1.000000  0.988506  0.994220
Average   0.997429  0.993680  0.995536


In [ ]:
Entrenamiento 
0.9925468096709689
1.0
Validación
0.7013081395348837
1.0
Predicción
1.0
1.0


In [35]:
## Ant: 1aAVb,RBBB,LBBB,SB,AF,ST
## CPSC: normal, AF,IAVB,LBBB,RBBB,PAC,PVC,STD,STE
predAnt = np.load("./dnn_predicts/pred_cpsc_antonizado.npy")
predAnt = np.delete(predAnt, [3,5],axis=1)
print(predAnt.shape)
predCorrects = pd.read_csv("./pred_CPSC_antonizado.csv")
predCorrects = predCorrects[["label_2","label_4","label_3","label_1"]]
print(predCorrects.shape)
thresholdAnt = pd.read_csv('dnn_predicts/optimal_thresholds_best_ant.csv')
resultAnt = pd.DataFrame(data= [])

for pred in predAnt:
    data = pd.DataFrame([pred >= thresholdAnt['Threshold']])
    resultAnt = pd.concat([resultAnt,data])
resultAnt = resultAnt.replace({True:1,False:0})

accuracyAnt = accuracy_score(predCorrects.values,resultAnt.values)
print(accuracyAnt)


(6877, 4)
(6877, 4)
0.8436818380107605


In [36]:
val_prec, val_recall, val_f1, _ = precision_recall_fscore_support(
    predCorrects, resultAnt
    , average = None
)

df2 = pd.DataFrame(np.array([val_prec,val_recall,val_f1]).transpose(),columns=["Precision","Recall","F1"],index=["I-AVB","LBBB","RBBB","AF"])
df2.loc['Average'] = df2.mean()
df2.index.names = ['Class']
print(df2)

         Precision    Recall        F1
Class                                 
I-AVB     0.823529  0.853186  0.838095
LBBB      0.952738  0.683899  0.796238
RBBB      0.824219  0.894068  0.857724
AF        0.955614  0.899263  0.926582
Average   0.889025  0.832604  0.854660


## comparar con nuestros casos

Caso1:STE
Caso2:I-AVB
Caso3:STE
Caso4:Normal
Caso5:PVC
Caso6:LBBB